In [1]:
import cv2
import numpy as np

#cap = cv2.VideoCapture(0) # webcam
cap = cv2.VideoCapture('Video/face_track.mp4')
face_casc = cv2.CascadeClassifier('HaarCascades/haarcascade_frontalface_default.xml') # cascade classifier

while True:
    ret, frame = cap.read()
    
    if ret==True:
        gray = cv2.cvtColor(frame,
                           cv2.COLOR_BGR2GRAY) # convert to gray
        
        face_rects = face_casc.detectMultiScale(gray, 
                                                1.3, 
                                                5)
        if face_rects is ():
            cv2.imshow('Facetracker', frame)
        else:
            face_x, face_y, w, h = tuple(face_rects[0])
            track_window = (face_x, face_y, w, h) # tracking window
            for (face_x, face_y, w, h) in face_rects:

                roi = frame[face_y:face_y+h,
                            face_x:face_x+w] # region of interest

                roi_hsv = cv2.cvtColor(roi,
                                      cv2.COLOR_BGR2HSV) # HSV color mapping

                # Histogram to target on each frame for meanshift calculation
                roi_hist = cv2.calcHist([roi_hsv],
                                        [0],
                                        None,
                                        [180],
                                        [0, 180])

                # Normalize the histogram
                cv2.normalize(roi_hist,
                              roi_hist,
                              0,
                              255,
                              cv2.NORM_MINMAX);

                term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

                # frame in hsv 
                hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

                # calculate the base of ROI
                dest_roi = cv2.calcBackProject([hsv],
                                              [0],
                                              roi_hist,
                                              [0, 180],
                                              1)

                # meanshift to calculate the new coordinates of the rectangle
                ret, track_window = cv2.meanShift(dest_roi,
                                                 track_window,
                                                 term_crit)

                # Draw new rectangle on image
                x,y,w,h = track_window

                img = cv2.rectangle(frame,
                                   (x,y),
                                   (x+w, y+h),
                                   (255,0,0),
                                   2)

                cv2.imshow('Facetracker',img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    else:
        break

cap.release()
cv2.destroyAllWindows()
print('Program Finished')

Program Finished
